<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0f4d0a24f039d16aac5c175d5b5f411af6c8dea542f1315ccab251a12f2c4703
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 13:01:41
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -49.10 K (-0.34%)
Current PnL: -23.79 L (-15.59%)
CY Booked + Current PnL: -9.90 L (-6.49%)
-------------------
Total profit:  1.22 L
Total loss:  -25.01 L
-------------------
Total Booked + Current PnL: 17.08 L (13.59%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.78%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.20 L (62.82%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.04,-14.19,16.64,0.09,14390.0,-14297.0,86480.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.62,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.02,-9.35,10.34,0.02,16361.0,-16329.0,158230.0,141.07,70.0,M-SC,6.67,234.0,-1.00,1.11,37.63,OX40N,NTT,PAINTS
43,ITC,452.00,-0.02,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.06,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.24,6.31,13.06,20.19,24203.0,11001.0,185321.0,-16.73,49.0,X-MC,7.56,68.0,0.45,1.31,22.38,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,-0.32,11.53,8.85,21.40,26199.0,30607.0,296033.0,3.53,52.0,X-LC,7.78,12.0,1.17,2.08,18.94,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,4.15,-49.25,233.09,69.05,157415.0,-65532.0,67534.0,-58.98,38.0,H-SC,23.54,140.0,-0.42,0.48,4.15,SR,ATH,IT
61,REPCOHOME,880.00,2.65,-10.69,112.53,89.82,287945.0,-30622.0,255883.0,-55.29,50.0,H-SC,10.43,160.0,-0.11,1.80,35.38,XY24,NTT,FINANCE
56,QUESS,424.00,2.45,-27.39,96.73,42.84,45656.0,-17807.0,47199.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.70,XY24,NTT,MISC
58,RAJOOENG,143.10,2.38,-29.13,97.00,39.61,70461.0,-29860.0,72640.0,-52.00,25.0,H-SC,20.03,135.0,-0.42,0.51,2.38,AR,ATH,MISC
64,SFL,1287.00,1.70,-40.18,111.68,26.63,175186.0,-105354.0,156864.0,18.85,31.0,M-SC,4.52,239.0,-0.60,1.10,1.70,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-3.62,-15.32,81.27,53.50,207653.0,-46241.0,255510.0,-59.04,45.0,X-MC,4.15,58.0,-0.22,1.80,13.49,XY24,NTT,FMCG
34,HONAUT,58357.33,-3.56,-14.10,66.85,43.32,93523.0,-22972.0,139900.0,-26.21,43.0,X-SC,2.01,90.0,-0.25,0.99,7.42,X40N,ATH,ELECTRICAL
24,EASEMYTRIP,26.40,-2.44,-60.21,266.16,45.70,187257.0,-106460.0,70355.0,-27.06,19.0,M-SC,21.76,205.0,-0.57,0.50,0.00,XY24,NTT,TRAVEL
3,ACC,3906.00,-2.36,-21.48,109.04,64.15,203741.0,-51101.0,186850.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.32,4.81,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.35,-63.89,177.11,0.07,87733.0,-87641.0,49536.0,1715.66,53.0,L-SC,1.54,268.0,-1.00,0.35,23.20,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,1.01,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,37.0,H-SC,3.74,139.0,0.0,0.87,23.57,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.04,-5.32,119.59,107.91,167177.0,-7852.0,139792.0,-22.98,32.0,M-SC,10.48,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.02,-9.35,10.34,0.02,16361.0,-16329.0,158230.0,141.07,70.0,M-SC,6.67,234.0,-1.00,1.11,37.63,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.04,-14.19,16.64,0.09,14390.0,-14297.0,86480.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.62,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.25,-22.77,46.93,13.47,97736.0,-61407.0,208260.0,-68.31,26.0,H-SC,1.39,158.0,-0.63,1.47,5.89,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.39,-23.50,30.71,-0.01,30024.0,-30040.0,97765.0,19900.00,54.0,M-SC,1.67,233.0,-1.00,0.69,10.70,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.33,1.09,70.41,72.27,120605.0,1842.0,171290.0,-13.34,55.0,M-LC,4.53,99.0,0.02,1.21,7.33,XR,NTT,IT
38,INDIAMART,4810.62,1.01,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,37.0,H-SC,3.74,139.0,0.00,0.87,23.57,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.33,1.09,70.41,72.27,120605.0,1842.0,171290.0,-13.34,55.0,M-LC,4.53,99.0,0.02,1.21,7.33,XR,NTT,IT
38,INDIAMART,4810.62,1.01,0.16,106.39,106.72,131426.0,196.0,123532.0,-50.44,37.0,H-SC,3.74,139.0,0.00,0.87,23.57,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.45,-1.84,38.17,35.63,78523.0,-3856.0,205718.0,-15.85,36.0,H-MC,4.39,119.0,-0.05,1.45,14.53,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.04,-5.32,119.59,107.91,167177.0,-7852.0,139792.0,-22.98,32.0,M-SC,10.48,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.00,-5.19,91.87,81.92,99996.0,-5953.0,108845.0,-44.42,32.0,M-SC,4.52,236.0,-0.06,0.77,28.86,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.03,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,21.0,X-LC,0.62,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-0.46,-9.79,27.73,15.23,50117.0,-19616.0,180731.0,-22.76,23.0,X-MC,6.66,66.0,-0.39,1.27,19.30,X40N,NTT,REALTY
11,BATAINDIA,2096.00,-0.50,-35.45,109.45,35.20,90907.0,-45612.0,83058.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.58,0.03,X40,NTT,FOOTWEAR
60,RELAXO,1176.00,-0.22,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,27.0,X-SC,3.96,91.0,-0.48,0.53,1.16,X40N,NTT,FOOTWEAR
22,DIXON,18931.72,-0.96,-6.70,30.31,21.58,39631.0,-9387.0,130752.0,-52.96,29.0,X-MC,6.86,56.0,-0.24,0.92,15.76,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.03,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,21.0,X-LC,0.62,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.02,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.06,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.07,0.61,30.17,30.97,53649.0,1084.0,177822.0,-23.09,46.0,X-MC,1.55,75.0,0.02,1.25,13.25,X40,ATH,INSURANCE
3,ACC,3906.0,-2.36,-21.48,109.04,64.15,203741.0,-51101.0,186850.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.32,4.81,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.28,-15.23,48.07,25.51,75830.0,-28345.0,157750.0,-3.08,52.0,H-LC,1.86,49.0,-0.37,1.11,16.25,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.61,-14.03,72.68,48.45,125286.0,-28126.0,172380.0,-20.30,29.0,X-MC,6.83,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.91,-5.94,34.02,26.07,52398.0,-9720.0,154020.0,-30.68,33.0,X-MC,9.06,55.0,-0.19,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-1.03,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,21.0,X-LC,0.62,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.50,-35.45,109.45,35.20,90907.0,-45612.0,83058.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.58,0.03,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.51,-4.61,40.22,33.76,77047.0,-9255.0,191565.0,-32.64,33.0,X-MC,3.37,57.0,-0.12,1.35,0.35,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.45,-27.39,96.73,42.84,45656.0,-17807.0,47199.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.70,XY24,NTT,MISC
60,RELAXO,1176.00,-0.22,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,27.0,X-SC,3.96,91.0,-0.48,0.53,1.16,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.50,-35.45,109.45,35.20,90907.0,-45612.0,83058.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.58,0.03,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.96,-6.70,30.31,21.58,39631.0,-9387.0,130752.0,-52.96,29.0,X-MC,6.86,56.0,-0.24,0.92,15.76,X40N,ATH,IT
52,MEDANTA,1486.00,-0.94,3.43,19.96,24.08,26208.0,4352.0,131302.0,-5.01,46.0,X-SC,4.94,89.0,0.17,0.92,23.72,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,-0.27,-14.88,40.14,19.28,118198.0,-51492.0,294464.0,-25.89,60.0,X-LC,7.52,1.0,-0.44,2.07,8.86,X40,ATH,IT
41,INFY,2275.00,-1.12,4.20,48.62,54.87,158520.0,13151.0,326039.0,-18.88,56.0,X-LC,3.01,2.0,0.08,2.30,12.83,X40,BTT,IT
76,TMPV,600.00,-1.03,-15.97,69.20,42.18,160726.0,-44147.0,232263.0,-24.30,21.0,X-LC,0.62,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.04,-9.85,50.49,35.67,143765.0,-31103.0,284739.0,-20.61,40.0,X-LC,5.96,4.0,-0.22,2.01,2.64,X40N,ATH,FMCG
43,ITC,452.00,-0.02,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.06,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.92,-12.44,107.48,81.67,88285.0,-11669.0,82141.0,7388.89,53.0,L-SC,18.79,271.0,-0.13,0.58,59.07,XR,NTT,CERAMICS
51,MASFIN,398.61,0.03,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,50.0,H-SC,7.34,164.0,-0.19,0.65,35.05,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.16,-19.96,53.71,23.03,76983.0,-35741.0,143330.0,55.70,48.0,H-SC,10.59,167.0,-0.46,1.01,42.21,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.02,-9.35,10.34,0.02,16361.0,-16329.0,158230.0,141.07,70.0,M-SC,6.67,234.0,-1.00,1.11,37.63,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.24,6.31,13.06,20.19,24203.0,11001.0,185321.0,-16.73,49.0,X-MC,7.56,68.0,0.45,1.31,22.38,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.02,-9.35,10.34,0.02,16361.0,-16329.0,158230.0,141.07,70.0,M-SC,6.67,234.0,-1.00,1.11,37.63,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.32,0.95,19.80,20.95,50326.0,2402.0,254170.0,-0.07,77.0,X-LC,13.40,31.0,0.05,1.79,36.93,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.12,-38.35,115.56,32.89,55004.0,-29608.0,47598.0,-687.51,59.0,L-MC,5.56,259.0,-0.54,0.34,31.08,XR,NTT,BANKS
12,BERGEPAINT,680.00,-0.58,2.60,18.07,21.14,42150.0,5921.0,233260.0,-4.59,61.0,X-MC,3.89,74.0,0.14,1.64,31.48,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,-0.50,4.61,15.45,20.77,37841.0,10784.0,244928.0,-4.44,57.0,X-MC,3.55,62.0,0.28,1.72,11.84,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,44.61
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.69
MC    29.20
LC    25.11
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.00
X40      21.18
X40N     11.85
XR        9.65
XY25      9.06
AR        8.98
OX40N     7.66
X200      1.84
SR        1.00
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    22.78
X-LC    19.98
M-SC    12.08
X-SC     8.23
H-MC     4.72
H-LC     2.95
L-SC     1.42
M-MC     1.36
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.76,-5.74,39.94
IT,13.74,-15.95,75.81
FINANCE,9.45,-15.51,64.31
MISC,7.42,-23.23,73.71
PAINTS,6.01,-8.13,24.19
ELECTRICAL,5.91,-11.83,52.79
INSURANCE,4.44,-0.27,35.61
PHARMA,3.88,-1.28,33.69
AUTO,3.33,-21.72,73.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3063123.0,21
AR,1297217.0,10
XR,1294885.0,13
X40,1028393.0,14
X40N,764357.0,9
OX40N,703204.0,10
XY25,353363.0,6
SR,279472.0,2
MH,73627.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3578941.0,25
M-SC,1344790.0,15
X-MC,1244547.0,16
X-LC,925777.0,11
X-SC,750344.0,8
H-MC,408143.0,3
L-SC,260056.0,3
H-LC,138432.0,2
M-LC,120605.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219948.0      6
           AR         879837.0      5
           XR         797335.0      7
M-SC       XY24       783352.0      6
X-MC       X40        450351.0      7
X-LC       X40        400820.0      5
X-MC       XY24       358174.0      3
H-SC       OX40N      328722.0      4
X-SC       X40N       297517.0      3
M-SC       OX40N      286749.0      5
H-SC       SR         279472.0      2
X-SC       XY24       275605.0      3
X-MC       X40N       252875.0      4
X-LC       XY24       234380.0      2
H-MC       AR         216479.0      2
X-LC       X40N       213965.0      2
H-MC       XY24       191664.0      1
X-MC       XY25       183147.0      2
X-SC       X40        177222.0      2
L-SC       XR         172323.0      2
M-SC       XR         149618.0      2
           AR         125071.0      2
M-LC       XR         120605.0      1
L-SC       OX40N       87733.0      1
X-LC       XY25        76612.0      2
H-LC       AR          75830.0      1
H-SC       MH          73627.0      1
H-LC       X200        62602.0      1
L-MC       XR          55004.0      1
M-MC       XY25        51585.0      1
L-LC       XY25        42019.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
